In [89]:
import math

import numpy as np
import pandas as pd

# Input parameters________________________________________________
L0 = 512
N_1 = 28
N_2 = 352
D_1 = L0*0.16
D_2 = L0*0.08

mu = 0.0240719927
rho = 1
Re = 100

V = L0**3
phi_1 = N_1*math.pi*(D_1)**3/(6.*V)
phi_2 = N_2*math.pi*(D_2)**3/(6.*V)
phi = phi_1 + phi_2
x_1 = phi_1/phi
x_2 = phi_2/phi
eps = 1 - phi
D = ((x_1/D_1)+(x_2/D_2))**(-1)
y_1 = D_1/D
y_2 = D_2/D

u_c = (Re * mu) / (rho * D)
F_conv = -0.5 * rho * (u_c**2) * ((np.pi * D**2) / 4)
F_st = -3 * np.pi * mu * u_c
F_c = F_conv
T_c = F_c * D

Ls = 0.033/L0
Ts = 1/10000
Ms = 1000/1*Ls**3

NN_data_final = np.array([])

# loading data________________________________________________________________
for kk in range(1,6):
    case = "case" + str(kk)
    mainpath = "Re" + str(Re) + "/" + str(N_1) + "-0p16-" + str(N_2) + "-0p08/" + case +"/initial_position.txt"
    # mainpath = str(N_1) + "-0p1-" + str(N_2) + "-0p05/" + case +"/initial_position.txt"
    datafile = pd.read_table(mainpath,delimiter=' ',skiprows=0)

    x_pos, y_pos, z_pos = datafile.iloc[:,0].to_numpy(),\
                          datafile.iloc[:,1].to_numpy(),\
                          datafile.iloc[:,2].to_numpy()

    r = np.zeros(np.size(x_pos))

    mainpath = "Re" + str(Re) + "/" + str(N_1) + "-0p16-" + str(N_2) + "-0p08/" + case +"/force-12000"
    datafile = pd.read_csv(mainpath,delimiter=' ',header=None)

    x_force, y_force, z_force = datafile.iloc[:,1].to_numpy(),\
                                datafile.iloc[:,2].to_numpy(),\
                                datafile.iloc[:,3].to_numpy()

    x_pos, y_pos, z_pos, r =   x_pos.reshape(-1,1),\
                               y_pos.reshape(-1,1),\
                               z_pos.reshape(-1,1),\
                               r.reshape(-1,1)

    x_force, y_force, z_force = x_force.reshape(-1,1),\
                                y_force.reshape(-1,1),\
                                z_force.reshape(-1,1)

# non-dimensionalization_____________________________________________________

    # x_pos, y_pos, z_pos =   x_pos / (D_2/512),\
    #                         y_pos / (D_2/512),\
    #                         z_pos / (D_2/512)

    for i in range(N_1):
        x_force[i], y_force[i], z_force[i], r[i] = x_force[i] * eps / F_st / D_1,\
                                                   y_force[i] * eps / F_st / D_1,\
                                                   z_force[i] * eps / F_st / D_1,\
                                                   D_1 / L0 /2
    
    for i in range(N_1,N_1+N_2):
        x_force[i], y_force[i], z_force[i], r[i] = x_force[i] * eps / F_st / D_2,\
                                                   y_force[i] * eps / F_st / D_2,\
                                                   z_force[i] * eps / F_st / D_2,\
                                                   D_2 / L0 /2
        
#     for i in range(N_1):
#         x_force[i], y_force[i], z_force[i], r[i] = 0,\
#                                                    0,\
#                                                    0,\
#                                                    D_1 / L0 /2
    
#     for i in range(N_1,N_1+N_2):
#         x_force[i], y_force[i], z_force[i], r[i] = 0,\
#                                                    0,\
#                                                    0,\
#                                                    D_2 / L0 /2
    
    # # printing information _____________________________________________________
    # F_d_F_st = x_force.mean() * F_c / F_st
    # print(f'F_d / F_st \t= \t{F_d_F_st:.2f}')
    # print(f'sigma_d / F_d \t= \t{(x_force.std() / x_force.mean()) * 100:.2f} %')
    # F_L = np.hstack((y_force, z_force))
    # F_L_mag = np.linalg.norm(F_L, axis=0)
    # print(f'F_L / F_st \t= \t{F_L.mean():.2g}')
    # ratio = F_L.std() / x_force.mean()
    # print(f'sigma_L / F_d \t= \t{ratio * 100:.2f} %')

    # Generate the data matrix with positions_______________________________
    xyzr = np.hstack((x_pos, y_pos, z_pos, r))

    # The extension of periodic mirror___________________________________
    L = 1
    delta = 0.5
    D = 0.16
    n_closest=10

    x_back  = xyzr[xyzr[:,0] > (L*delta + D), 0].reshape(-1,1) - L
    x_front = xyzr[xyzr[:,0] < (L*delta + D), 0].reshape(-1,1) + L

    y_back  = xyzr[xyzr[:,1] > (L*delta + D), 1].reshape(-1,1) - L
    y_front = xyzr[xyzr[:,1] < (L*delta + D), 1].reshape(-1,1) + L

    z_back  = xyzr[xyzr[:,2] > (L*delta + D), 2].reshape(-1,1) - L
    z_front = xyzr[xyzr[:,2] < (L*delta + D), 2].reshape(-1,1) + L

    xyzr_x_back    = np.hstack((x_back, y_pos[xyzr[:,0] > (L*delta + D)], z_pos[xyzr[:,0] > (L*delta + D)], r[xyzr[:,0] > (L*delta + D)]))
    xyzr_x_front   = np.hstack((x_front, y_pos[xyzr[:,0] < (L*delta + D)], z_pos[xyzr[:,0] < (L*delta + D)], r[xyzr[:,0] < (L*delta + D)]))

    xyzr_y_back    = np.hstack((x_pos[xyzr[:,1] > (L*delta + D)], y_back, z_pos[xyzr[:,1] > (L*delta + D)], r[xyzr[:,1] > (L*delta + D)]))
    xyzr_y_front   = np.hstack((x_pos[xyzr[:,1] < (L*delta + D)], y_front, z_pos[xyzr[:,1] < (L*delta + D)], r[xyzr[:,1] < (L*delta + D)]))

    xyzr_z_back    = np.hstack((x_pos[xyzr[:,2] > (L*delta + D)], y_pos[xyzr[:,2] > (L*delta + D)], z_back, r[xyzr[:,2] > (L*delta + D)]))
    xyzr_z_front   = np.hstack((x_pos[xyzr[:,2] < (L*delta + D)], y_pos[xyzr[:,2] < (L*delta + D)], z_front, r[xyzr[:,2] < (L*delta + D)]))

    num_p = np.size(xyzr, axis=0)

    # here, I add the periodic images to the original xyzr to be considered in the distances
    xyzr = np.vstack((xyzr, xyzr_x_back, xyzr_x_front, xyzr_y_back, xyzr_y_front, xyzr_z_back, xyzr_z_front))
    # the distance loop should be done over periodic particles as well
    num_p_periodic = np.size(xyzr, axis=0)

    NN_data = np.zeros( (num_p, 4 * (n_closest - 1) + 10 ) )
    
    for m in np.arange(num_p):

        # Just a progress indicator
        if m%100 == 0 and m!=0:
            print('Particle '+str(m)+' is done!')

        dist  = np.zeros( (num_p_periodic, 5) )

        for i in range(num_p_periodic):
            dist[i,0] = np.sqrt( ( xyzr[i, 0] - xyzr[m, 0] )**2 +
                                 ( xyzr[i, 1] - xyzr[m, 1] )**2 +
                                 ( xyzr[i, 2] - xyzr[m, 2] )**2 )
            dist[i,1] = xyzr[i, 0] - xyzr[m, 0]
            dist[i,2] = xyzr[i, 1] - xyzr[m, 1]
            dist[i,3] = xyzr[i, 2] - xyzr[m, 2]
            dist[i,4] = xyzr[i, 3]
            
        dist = dist[ np.argsort( dist[:, 0] ) ]
        dist = dist[:n_closest, :]
        dist = dist[1:, 1:]
        dist = dist.flatten()
        dist = dist.reshape(1, -1)

        NN_data[m,:dist.size] = dist
        
        NN_data[m,-10] = xyzr[m, 3]

        NN_data[m,-9] = 0
        NN_data[m,-8] = 0
        NN_data[m,-7] = 0
    
        NN_data[m,-6] = 0
        NN_data[m,-5] = 0
        NN_data[m,-4] = 0
    
        NN_data[m,-3] = x_force[m, 0]
        NN_data[m,-2] = y_force[m, 0]
        NN_data[m,-1] = z_force[m, 0]
        
    if kk == 1:
        NN_data_final = NN_data
    else:
        NN_data_final = np.vstack((NN_data_final, NN_data))

np.savetxt("new_3D_Re100_phi015_M=200", NN_data_final)

Particle 100 is done!
Particle 200 is done!
Particle 300 is done!
Particle 100 is done!
Particle 200 is done!
Particle 300 is done!
Particle 100 is done!
Particle 200 is done!
Particle 300 is done!
Particle 100 is done!
Particle 200 is done!
Particle 300 is done!
Particle 100 is done!
Particle 200 is done!
Particle 300 is done!


## Local volume fraction ratio

In [ ]:
import math

import numpy as np
import pandas as pd

L0 = 512
Re = 10
N_1 = 192
if N_1 == 192: 
    N_2 = 768
    N_set = 20
    D_1 = L0*0.1
    D_2 = L0*0.05
    c = 4*D_2
elif N_1 == 384:
    N_2 = 1536
    N_set = 10
    D_1 = L0*0.1
    D_2 = L0*0.05
    c = 3*D_2
elif N_1 == 313:
    N_2 = 1222
    N_set = 13
    D_1 = L0*0.125
    D_2 = L0*0.05
    c = 3*D_2
elif N_1 == 453:
    N_2 = 1528
    N_set = 10
    D_1 = L0*0.075
    D_2 = L0*0.05
    c = 4*D_2
elif N_1 == 288:
    N_2 = 1152
    N_set = 10
    D_1 = L0*0.1
    D_2 = L0*0.05
    c = 4*D_2
elif N_1 == 393:
    N_2 = 1528
    N_set = 10
    D_1 = L0*0.09
    D_2 = L0*0.05
    c = 4*D_2
case_N = 2

print(D_1,D_2)

# for kk in range(case_N,case_N+10):
for kk in range(case_N,case_N+1):
    '''
    Reading data from files:
    x,y,z_pos          - Coordinates of sphere centers
    x,y,z_force/torque - Forces and torques on each sphere
    u,v,w_avg          - Local average velocities
    phi_local          - Local volume fraction
    '''
    case = "case" + str(kk)
    mainpath = "Uni/Re" + str(Re) + "/" + str(N_1) + "-" + str(N_2) + "/" + case +"/initial_position.txt"
    datafile = pd.read_table(mainpath,delimiter=' ',skiprows=0)

    x_pos, y_pos, z_pos, r = datafile.iloc[:,0].to_numpy()*L0,\
                             datafile.iloc[:,1].to_numpy()*L0,\
                             datafile.iloc[:,2].to_numpy()*L0,\
                             datafile.iloc[:,3].to_numpy()*L0
    
    N = x_pos.shape[0]
    v_ratio = np.zeros(N)
    for i in range(N):
        if r[i] == np.max(r):
            V_large = r[i]**3
            V_small = 0
        else:
            V_large = 0
            V_small = r[i]**3
        target = (x_pos[i], y_pos[i], z_pos[i])
        # print(i, x_pos[i], y_pos[i], z_pos[i]) 
        for j in range(N):
            if j != i:
                local_dist = 1e10
                for ii in [-1,0,1]:
                    for jj in [-1,0,1]:
                        for kk in [-1,0,1]:
                            neighbor = (x_pos[j]+ii*L0, y_pos[j]+jj*L0, z_pos[j]+kk*L0)
                            local_dist_temp = math.dist(target,neighbor)
                            if local_dist_temp < local_dist:
                                local_dist = local_dist_temp
                if local_dist <= c:
                    # print(j, x_pos[j], y_pos[j], z_pos[j], local_dist)
                    if r[j] == np.max(r):
                        V_large += r[j]**3
                    else:
                        V_small += r[j]**3
        v_ratio[i] = V_large/V_small
        print(i,v_ratio[i])
print(np.mean(v_ratio[:192]),np.mean(v_ratio[192:]))

## Normalization

In [6]:
import math

import numpy as np
import pandas as pd

'''
Input parameters: all parameters are in LB units.
L0  - Box size
N_1 - Number of large sphere in one realization
N_2 - Number of small sphere in one realization
D_1 - Large sphere diameter
D_2 - small sphere diameter
mu  - Dluid dynamic viscosity
rho - Fluid density
Re  - Reynolds number
'''
L0 = 512
N_1 = 384
if N_1 == 192: 
    N_2 = 768
    N_set = 20
    D_1 = L0*0.1
    D_2 = L0*0.05
    c_l = 3*D_2
    c_s = 2.5*D_2
elif N_1 == 384:
    N_2 = 1536
    N_set = 10
    D_1 = L0*0.1
    D_2 = L0*0.05
    c_l = 3*D_2
    c_s = 2.5*D_2
elif N_1 == 313:
    N_2 = 1222
    N_set = 13
    D_1 = L0*0.125
    D_2 = L0*0.05
    c_l = 5*D_2
    c_s = 3*D_2
elif N_1 == 453:
    N_2 = 1528
    N_set = 10
    D_1 = L0*0.075
    D_2 = L0*0.05
    c_l = 5*D_2
    c_s = 3.5*D_2
elif N_1 == 288:
    N_2 = 1152
    N_set = 10
    D_1 = L0*0.1
    D_2 = L0*0.05
    c_l = 5*D_2
    c_s = 3.5*D_2
elif N_1 == 393:
    N_2 = 1528
    N_set = 10
    D_1 = L0*0.09
    D_2 = L0*0.05
    c_l = 5*D_2
    c_s = 3.5*D_2

rho = 1
phi_sys = 30
Re = 1
if Re == 100:
    mu = 0.0131072
elif Re == 40:
    mu = 0.02048
elif Re == 20:
    mu = 0.0524288
elif Re == 10:
    mu = 0.08192
elif Re == 5:
    mu = 0.2097152    
else:
    mu = 0.32768
    
V = L0**3
phi_1 = N_1*math.pi*(D_1)**3/(6.*V)
phi_2 = N_2*math.pi*(D_2)**3/(6.*V)
phi = phi_1 + phi_2
x_1 = phi_1/phi
x_2 = phi_2/phi
eps = 1 - phi
D_eql = ((x_1/D_1)+(x_2/D_2))**(-1)
U_c = (Re * mu) / (rho * D_eql)
y_1 = D_1/D_eql
y_2 = D_2/D_eql

print("Re                       = {:>12.8f}".format(Re))
print("Fluid Volume Fraction    = {:>12.8f}".format(eps))
print("Equivalent Diameter      = {:>12.8f}".format(D_eql))
print("x1                       = {:>12.8f}".format(x_1))
print("x2                       = {:>12.8f}".format(x_2))
print("y1/<y>                   = {:>12.8f}".format(y_1))
print("y2/<y>                   = {:>12.8f}".format(y_2))
print("Uc                       = {:>12.8f}".format(U_c))

NN_data_final = np.array([])

case_N = 1
n_closest=101
origin = (0,0,0)

# for kk in range(case_N,case_N+10):
for kk in range(case_N,N_set+1):
    '''
    Reading data from files:
    x,y,z_pos          - Coordinates of sphere centers
    x,y,z_force/torque - Forces and torques on each sphere
    u,v,w_avg          - Local average velocities
    phi_local          - Local volume fraction
    '''
    case = "case" + str(kk)
    mainpath = "Uni/Re" + str(Re) + "/" + str(N_1) + "-" + str(N_2) + "/" + case +"/initial_position.txt"
    datafile = pd.read_table(mainpath,delimiter=' ',skiprows=0)

    x_pos, y_pos, z_pos, r = datafile.iloc[:,0].to_numpy()*L0,\
                             datafile.iloc[:,1].to_numpy()*L0,\
                             datafile.iloc[:,2].to_numpy()*L0,\
                             datafile.iloc[:,3].to_numpy()*L0
    
    #============================volume fraction ratio================================
    N = x_pos.shape[0]
    v_ratio = np.zeros(N)
    V_large = np.zeros(N)
    V_small = np.zeros(N)
    
    for i in range(N):
        if r[i] == np.max(r):
            V_large[i] = 0#r[i]**3
            V_small[i] = 0
            c = c_l
        else:
            V_large[i] = 0
            V_small[i] = 0#r[i]**3
            c = c_s
        target = (x_pos[i], y_pos[i], z_pos[i])
        # print(i, x_pos[i], y_pos[i], z_pos[i]) 
        for j in range(N):
            if j != i:
                local_dist = 1e10
                for iii in [-1,0,1]:
                    for jjj in [-1,0,1]:
                        for kkk in [-1,0,1]:
                            neighbor = (x_pos[j]+iii*L0, y_pos[j]+jjj*L0, z_pos[j]+kkk*L0)
                            local_dist_temp = math.dist(target,neighbor)
                            if local_dist_temp < local_dist:
                                local_dist = local_dist_temp
                if local_dist <= c-r[j]:
                    # print(j, x_pos[j], y_pos[j], z_pos[j], local_dist)
                    if r[j] == np.max(r):
                        V_large[i] += r[j]**3
                    else:
                        V_small[i] += r[j]**3
                elif local_dist <= c+r[j]:
                    if r[j] == np.max(r):
                        V_large[i] += r[j]**3*abs(c-local_dist+r[j])/(2*r[j])
                    else:
                        V_small[i] += r[j]**3*abs(c-local_dist+r[j])/(2*r[j])
        # v_ratio[i] = V_large/V_small
        V_large[i] /= c**3
        V_small[i] /= c**3
    #============================volume fraction ratio================================

    mainpath = "Uni/Re" + str(Re) + "/" + str(N_1) + "-" + str(N_2) + "/" + case +"/force-local"
    datafile = pd.read_csv(mainpath,delimiter=' ',header=None)

    x_force, y_force, z_force = datafile.iloc[:,1].to_numpy(),\
                                datafile.iloc[:,2].to_numpy(),\
                                datafile.iloc[:,3].to_numpy()
    
    x_torque, y_torque, z_torque = datafile.iloc[:,4].to_numpy(),\
                                   datafile.iloc[:,5].to_numpy(),\
                                   datafile.iloc[:,6].to_numpy()
    
    u_avg, v_avg, w_avg = datafile.iloc[:,7].to_numpy(),\
                          datafile.iloc[:,8].to_numpy(),\
                          datafile.iloc[:,9].to_numpy()
    
    phi_local = datafile.iloc[:,10].to_numpy()
    
    '''
    reshape to have an array with one column:
    '''
    x_pos, y_pos, z_pos, r =   x_pos.reshape(-1,1),\
                               y_pos.reshape(-1,1),\
                               z_pos.reshape(-1,1),\
                               r.reshape(-1,1)

    x_force, y_force, z_force = x_force.reshape(-1,1),\
                                y_force.reshape(-1,1),\
                                z_force.reshape(-1,1)
    
    x_torque, y_torque, z_torque = x_torque.reshape(-1,1),\
                                   y_torque.reshape(-1,1),\
                                   z_torque.reshape(-1,1)
    
    phi_local = phi_local.reshape(-1,1)
    eps_local = np.ones((phi_local.shape[0],1))-phi_local
    
    u_avg, v_avg, w_avg = u_avg.reshape(-1,1)*eps_local,\
                          v_avg.reshape(-1,1)*eps_local,\
                          w_avg.reshape(-1,1)*eps_local
    
    

    '''
    Non-dimensionalization:
    Length scale   - L_scale = D_2
    Velocity scale - V_scale = mu/(rho*D_i)
    Force scale    - F_scale = 3*pi*mu*D_i*|u|
    Torque scale   - T_scale = mu*D_i^2*|u|
    '''
    for i in range(N_1+N_2):
        V_vec = (u_avg[i], v_avg[i], w_avg[i])
        V_mag = math.dist(V_vec,origin)
        V_scale = mu/(2*r[i])#(1*mu)/(rho*D_eql)#
        F_scale = -3*math.pi*mu*r[i]*2*V_mag
        T_scale = -mu*(r[i]*2)**2*V_mag
        # V_scale = U_c
        # # F_scale = -0.5 * rho * (U_c**2) * (np.pi * r[i]**2)
        # F_scale = -3*math.pi*mu*r[i]*2*U_c
        # T_scale = F_scale*(r[i]*2)
        x_force[i], y_force[i], z_force[i] = x_force[i] / F_scale,\
                                             y_force[i] / F_scale,\
                                             z_force[i] / F_scale,\

        x_torque[i], y_torque[i], z_torque[i] = x_torque[i] / T_scale,\
                                                y_torque[i] / T_scale,\
                                                z_torque[i] / T_scale
        
        u_avg[i], v_avg[i], w_avg[i] = u_avg[i] / V_scale,\
                                       v_avg[i] / V_scale,\
                                       w_avg[i] / V_scale,\

    x_pos, y_pos, z_pos, r = x_pos / (D_2),\
                             y_pos / (D_2),\
                             z_pos / (D_2),\
                             r / (D_2)

    '''
    Generate the data matrix with positions:
    '''
    xyzr = np.hstack((x_pos, y_pos, z_pos, r))
    num_p = np.size(xyzr, axis=0)

    '''
    The extension of periodic mirror:
    '''
    L = L0/D_2
    delta = 0.5
    D = 2
    
    xyzr_periodic = np.zeros([num_p,4,26])
    
    periodic_index = 0
    for i in [-1,0,1]:
        for j in [-1,0,1]:
            for k in [-1,0,1]:
                if ((i == 0 and j == 0 and k == 0)==False):
                    xyzr_periodic[:,:,periodic_index] = xyzr+np.tile([i,j,k,0],[num_p,1])*L
                    periodic_index += 1

#     x_back  = xyzr[xyzr[:,0] > (L*delta + D), 0].reshape(-1,1) - L
#     x_front = xyzr[xyzr[:,0] < (L*delta + D), 0].reshape(-1,1) + L

#     y_back  = xyzr[xyzr[:,1] > (L*delta + D), 1].reshape(-1,1) - L
#     y_front = xyzr[xyzr[:,1] < (L*delta + D), 1].reshape(-1,1) + L

#     z_back  = xyzr[xyzr[:,2] > (L*delta + D), 2].reshape(-1,1) - L
#     z_front = xyzr[xyzr[:,2] < (L*delta + D), 2].reshape(-1,1) + L

#     xyzr_x_back    = np.hstack((x_back, y_pos[xyzr[:,0] > (L*delta + D)], z_pos[xyzr[:,0] > (L*delta + D)], r[xyzr[:,0] > (L*delta + D)]))
#     xyzr_x_front   = np.hstack((x_front, y_pos[xyzr[:,0] < (L*delta + D)], z_pos[xyzr[:,0] < (L*delta + D)], r[xyzr[:,0] < (L*delta + D)]))

#     xyzr_y_back    = np.hstack((x_pos[xyzr[:,1] > (L*delta + D)], y_back, z_pos[xyzr[:,1] > (L*delta + D)], r[xyzr[:,1] > (L*delta + D)]))
#     xyzr_y_front   = np.hstack((x_pos[xyzr[:,1] < (L*delta + D)], y_front, z_pos[xyzr[:,1] < (L*delta + D)], r[xyzr[:,1] < (L*delta + D)]))

#     xyzr_z_back    = np.hstack((x_pos[xyzr[:,2] > (L*delta + D)], y_pos[xyzr[:,2] > (L*delta + D)], z_back, r[xyzr[:,2] > (L*delta + D)]))
#     xyzr_z_front   = np.hstack((x_pos[xyzr[:,2] < (L*delta + D)], y_pos[xyzr[:,2] < (L*delta + D)], z_front, r[xyzr[:,2] < (L*delta + D)]))

    '''
    Add periodic boxes to the original domain
    '''
    for i in range(26):
        xyzr = np.vstack((xyzr,xyzr_periodic[:,:,i]))
        
    xyzr = xyzr[xyzr[:,0]>(D-L/3)]
    xyzr = xyzr[xyzr[:,0]<(D+4*L/3)]
    xyzr = xyzr[xyzr[:,1]>(D-L/3)]
    xyzr = xyzr[xyzr[:,1]<(D+4*L/3)]
    xyzr = xyzr[xyzr[:,2]>(D-L/3)]
    xyzr = xyzr[xyzr[:,2]<(D+4*L/3)]
        
    '''
    The distance loop should be done over periodic particles as well
    '''
    num_p_periodic = np.size(xyzr, axis=0)

    NN_data = np.zeros( (num_p, 4 * (n_closest - 1) + 12 ) )
    
    for m in np.arange(num_p):

        # Just a progress indicator
        if m%100 == 0 and m!=0:
            print('Particle '+str(m)+' is done!')

        dist  = np.zeros( (num_p_periodic, 5) )

        for i in range(num_p_periodic):
            dist[i,0] = np.sqrt( ( xyzr[i, 0] - xyzr[m, 0] )**2 +
                                 ( xyzr[i, 1] - xyzr[m, 1] )**2 +
                                 ( xyzr[i, 2] - xyzr[m, 2] )**2 )
            dist[i,1] = xyzr[i, 0] - xyzr[m, 0]
            dist[i,2] = xyzr[i, 1] - xyzr[m, 1]
            dist[i,3] = xyzr[i, 2] - xyzr[m, 2]
            dist[i,4] = xyzr[i, 3]
            
        dist = dist[ np.argsort( dist[:, 0] ) ]
        dist = dist[:n_closest, :]
        dist = dist[1:, 1:]
        dist = dist.flatten()
        dist = dist.reshape(1, -1)

        NN_data[m,:dist.size] = dist
        
        NN_data[m,-12] = xyzr[m, 3]
        NN_data[m,-11] = V_large[m]#phi_local[m]*v_ratio[m]/(v_ratio[m]+1)
        NN_data[m,-10] = V_small[m]#phi_local[m]/(v_ratio[m]+1)

        NN_data[m,-9] = u_avg[m]
        NN_data[m,-8] = v_avg[m]
        NN_data[m,-7] = w_avg[m]
    
        NN_data[m,-6] = x_torque[m]
        NN_data[m,-5] = y_torque[m]
        NN_data[m,-4] = z_torque[m]
    
        NN_data[m,-3] = x_force[m]
        NN_data[m,-2] = y_force[m]
        NN_data[m,-1] = z_force[m]
        
    if kk == case_N:
        NN_data_final = NN_data
    else:
        NN_data_final = np.vstack((NN_data_final, NN_data))
        
    print('=========================')    
    print('Case '+str(kk)+' is done!')
    print('=========================') 

np.savetxt(f"Datasets/Full_c_Re{Re}_phi0{phi_sys}_M={n_closest}_{N_1}L", NN_data_final)

Re                       =   1.00000000
Fluid Volume Fraction    =   0.69840711
Equivalent Diameter      =  38.40000000
x1                       =   0.66666667
x2                       =   0.33333333
y1/<y>                   =   1.33333333
y2/<y>                   =   0.66666667
Uc                       =   0.00853333
Particle 100 is done!
Particle 200 is done!
Particle 300 is done!
Particle 400 is done!
Particle 500 is done!
Particle 600 is done!
Particle 700 is done!
Particle 800 is done!
Particle 900 is done!
Particle 1000 is done!
Particle 1100 is done!
Particle 1200 is done!
Particle 1300 is done!
Particle 1400 is done!
Particle 1500 is done!
Particle 1600 is done!
Particle 1700 is done!
Particle 1800 is done!
Particle 1900 is done!
Case 1 is done!
Particle 100 is done!
Particle 200 is done!
Particle 300 is done!
Particle 400 is done!
Particle 500 is done!
Particle 600 is done!
Particle 700 is done!
Particle 800 is done!
Particle 900 is done!
Particle 1000 is done!
Particle 1100 i

In [75]:
# a = np.array([[2,3,4],
#               [5,6,7]])
# print(a[::1,:np.size(a,axis=1)-1])
v = np.array([[1,2],[3,4]])
a = 0
for i in [-1,0,1]:
    for j in [-1,0,1]:
        for k in [-1,0,1]:
            if ((i == 0 and j == 0 and k == 0)==False):
                b = np.tile([i,j,k],[3,1])
                a += 1
                print(b,a)

[[-1 -1 -1]
 [-1 -1 -1]
 [-1 -1 -1]] 1
[[-1 -1  0]
 [-1 -1  0]
 [-1 -1  0]] 2
[[-1 -1  1]
 [-1 -1  1]
 [-1 -1  1]] 3
[[-1  0 -1]
 [-1  0 -1]
 [-1  0 -1]] 4
[[-1  0  0]
 [-1  0  0]
 [-1  0  0]] 5
[[-1  0  1]
 [-1  0  1]
 [-1  0  1]] 6
[[-1  1 -1]
 [-1  1 -1]
 [-1  1 -1]] 7
[[-1  1  0]
 [-1  1  0]
 [-1  1  0]] 8
[[-1  1  1]
 [-1  1  1]
 [-1  1  1]] 9
[[ 0 -1 -1]
 [ 0 -1 -1]
 [ 0 -1 -1]] 10
[[ 0 -1  0]
 [ 0 -1  0]
 [ 0 -1  0]] 11
[[ 0 -1  1]
 [ 0 -1  1]
 [ 0 -1  1]] 12
[[ 0  0 -1]
 [ 0  0 -1]
 [ 0  0 -1]] 13
[[0 0 1]
 [0 0 1]
 [0 0 1]] 14
[[ 0  1 -1]
 [ 0  1 -1]
 [ 0  1 -1]] 15
[[0 1 0]
 [0 1 0]
 [0 1 0]] 16
[[0 1 1]
 [0 1 1]
 [0 1 1]] 17
[[ 1 -1 -1]
 [ 1 -1 -1]
 [ 1 -1 -1]] 18
[[ 1 -1  0]
 [ 1 -1  0]
 [ 1 -1  0]] 19
[[ 1 -1  1]
 [ 1 -1  1]
 [ 1 -1  1]] 20
[[ 1  0 -1]
 [ 1  0 -1]
 [ 1  0 -1]] 21
[[1 0 0]
 [1 0 0]
 [1 0 0]] 22
[[1 0 1]
 [1 0 1]
 [1 0 1]] 23
[[ 1  1 -1]
 [ 1  1 -1]
 [ 1  1 -1]] 24
[[1 1 0]
 [1 1 0]
 [1 1 0]] 25
[[1 1 1]
 [1 1 1]
 [1 1 1]] 26


## Average and Standard Deviation

In [1]:
import math

import numpy as np
import pandas as pd

N_1 = 192
N_2 = 768

L = 512
D_1 = L*0.1
D_2 = L*0.05
NOR = 20

V = L**3
phi_1 = N_1*math.pi*(D_1)**3/(6.*V)
phi_2 = N_2*math.pi*(D_2)**3/(6.*V)
phi = phi_1 + phi_2
x_1 = phi_1/phi
x_2 = phi_2/phi
eps = 1 - phi
D = ((x_1/D_1)+(x_2/D_2))**(-1)
y_1 = D_1/D
y_2 = D_2/D

Re = 100
if Re == 100:
    nu = 0.0131072
else:
    nu = 0.32768
u  = Re * nu / D

num_par = (N_1+N_2)*NOR
F_l = np.zeros((N_1*NOR,6))
F_s = np.zeros((N_2*NOR,6))
T_l = np.zeros((N_1*NOR,6))
T_s = np.zeros((N_2*NOR,6))

F_conv_1 = -0.5 * (u**2) * ((np.pi * D_1**2) / 4)
F_conv_2 = -0.5 * (u**2) * ((np.pi * D_2**2) / 4)
F_st = -3 * np.pi * nu * u 
F_c_1 = -3 * np.pi * nu * u * D_1
F_c_2 = -3 * np.pi * nu * u * D_2
T_c_1 = F_c_1 * D_1
T_c_2 = F_c_2 * D_2

for j in range(1,NOR+1):
    case = "case" + str(j)
    mainpath = "Uni/Re" + str(Re) + "/" + str(N_1) + "-" + str(N_2) + "/" + case +"/force-local"

    # #species 1
    # for i in range(0,N_1):
    datafile = pd.read_table(mainpath,delimiter=' ',header=None).to_numpy()
    F_l[(j-1)*N_1:j*N_1,0:3] = datafile[0:N_1,1:4]/F_c_1
    T_l[(j-1)*N_1:j*N_1,0:3] = datafile[0:N_1,4:7]/T_c_1
    F_s[(j-1)*N_2:j*N_2,0:3] = datafile[N_1:N_1+N_2,1:4]/F_c_2
    T_s[(j-1)*N_2:j*N_2,0:3] = datafile[N_1:N_1+N_2,4:7]/T_c_2

print(T_s.shape[0])
# T_l = T_l[abs(T_l[:,2])<0.8]
# T_s = T_s[abs(T_s[:,2])<0.6]
print(T_s.shape[0])

print("=============Mean values=============")
print("Large particle F_x mean  = {:>10.6f}".format(np.mean(F_l[:,0])))
print("Large particle F_y mean  = {:>10.6f}".format(np.mean(F_l[:,1])))
print("Large particle F_z mean  = {:>10.6f}".format(np.mean(F_l[:,2])))
print("Large particle T_x mean  = {:>10.6f}".format(np.mean(T_l[:,0])))
print("Large particle T_y mean  = {:>10.6f}".format(np.mean(T_l[:,1])))
print("Large particle T_z mean  = {:>10.6f}".format(np.mean(T_l[:,2])))
print("Small particle F_x mean  = {:>10.6f}".format(np.mean(F_s[:,0])))
print("Small particle F_y mean  = {:>10.6f}".format(np.mean(F_s[:,1])))
print("Small particle F_z mean  = {:>10.6f}".format(np.mean(F_s[:,2])))
print("Small particle T_x mean  = {:>10.6f}".format(np.mean(T_s[:,0])))
print("Small particle T_y mean  = {:>10.6f}".format(np.mean(T_s[:,1])))
print("Small particle T_z mean  = {:>10.6f}".format(np.mean(T_s[:,2])))
print("=========Standard Deviations=========")
print("Large particle F_x std   = {:>10.6f}".format(np.std(F_l[:,0])))
print("Large particle F_y std   = {:>10.6f}".format(np.std(F_l[:,1])))
print("Large particle F_z std   = {:>10.6f}".format(np.std(F_l[:,2])))
print("Large particle T_x std   = {:>10.6f}".format(np.std(T_l[:,0])))
print("Large particle T_y std   = {:>10.6f}".format(np.std(T_l[:,1])))
print("Large particle T_z std   = {:>10.6f}".format(np.std(T_l[:,2])))
print("Small particle F_x std   = {:>10.6f}".format(np.std(F_s[:,0])))
print("Small particle F_y std   = {:>10.6f}".format(np.std(F_s[:,1])))
print("Small particle F_z std   = {:>10.6f}".format(np.std(F_s[:,2])))
print("Small particle T_x std   = {:>10.6f}".format(np.std(T_s[:,0])))
print("Small particle T_y std   = {:>10.6f}".format(np.std(T_s[:,1])))
print("Small particle T_z std   = {:>10.6f}".format(np.std(T_s[:,2])))

15360
15360
=============Mean values=============
Large particle F_x mean  =  15.089332
Large particle F_y mean  =  -0.042260
Large particle F_z mean  =   0.003258
Large particle T_x mean  =   0.000199
Large particle T_y mean  =  -0.002075
Large particle T_z mean  =   0.003912
Small particle F_x mean  =   7.567062
Small particle F_y mean  =   0.018419
Small particle F_z mean  =  -0.000693
Small particle T_x mean  =  -0.000033
Small particle T_y mean  =  -0.000994
Small particle T_z mean  =  -0.000896
=========Standard Deviations=========
Large particle F_x std   =   3.135263
Large particle F_y std   =   2.106473
Large particle F_z std   =   2.136747
Large particle T_x std   =   0.065781
Large particle T_y std   =   0.207591
Large particle T_z std   =   0.204546
Small particle F_x std   =   2.510142
Small particle F_y std   =   1.421573
Small particle F_z std   =   1.415807
Small particle T_x std   =   0.050616
Small particle T_y std   =   0.144847
Small particle T_z std   =   0.143566
